In [ ]:
import numpy as np
import glob2
import datetime
from pathlib import Path
from tqdm.notebook import tqdm
import pickle
from matplotlib import pyplot as plt

from utils.detection.association_geodesic import squarize
import matplotlib as mpl
import matplotlib.dates as mdates
import matplotlib.ticker as mticker

from utils.physics.geodesic.distance import distance_point_point
from utils.physics.signal.make_spectrogram import make_spectrogram

plt.style.use('classic')
plt.rcParams.update({
    "pgf.texsystem": "pdflatex",
    "text.usetex": True,
    "font.family": "serif",
    "font.size": 10,
    "axes.titlesize": 10,
    "axes.labelsize": 10,
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
    "legend.fontsize": 8,
})
from matplotlib import rc
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)
import math
from numpy.linalg import LinAlgError
import pandas as pd
import cartopy.feature as cfeature
import matplotlib.patches as mpatches
import matplotlib.gridspec as gridspec
import cartopy.crs as ccrs
from collections import Counter
from matplotlib.ticker import ScalarFormatter
from utils.data_reading.sound_data.station import StationsCatalog
from utils.physics.sound_model.spherical_sound_model import GridSphericalSoundModel as GridSoundModel, MonthlyHomogeneousSphericalSoundModel as HomogeneousSoundModel
from utils.detection.association_geodesic import compute_candidates, update_valid_grid, update_results, load_detections, compute_grids

In [ ]:
CATALOG_PATH = "/media/plerolland/akoustik"
STATIONS = StationsCatalog(CATALOG_PATH).filter_out_undated().filter_out_unlocated()

s2 = STATIONS.by_name("MAHY32")[0]
s3 = STATIONS.by_name("MAHY33")[0]
s4 = STATIONS.by_name("MAHY34")[0]
date2 = datetime.datetime(2023, 1, 16, 6, 45, 50, 500000)
date3 = datetime.datetime(2023, 1, 16, 6, 45, 31, 500000)
date4 = datetime.datetime(2023, 1, 16, 6, 44, 51, 500000)
DELTA_S = 30

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(5.5, 3), facecolor='white')
width_in = 5.5
height_in = width_in / 1.08
fig.set_size_inches(width_in, height_in)

m = s2.get_manager()
data = m.get_segment(date2 - datetime.timedelta(seconds=DELTA_S), date2 + datetime.timedelta(seconds=DELTA_S))
f_EQ, t_EQ, spectro_EQ = make_spectrogram(
    data, m.sampling_f, t_res=0.25, f_res=1,
    return_bins=True, normalize=True, vmin=60, vmax=120
)
ax = axs[0]
im0 = ax.imshow(
    spectro_EQ[::-1], cmap="inferno", aspect="auto", origin="lower",
    extent=(-DELTA_S, DELTA_S, f_EQ[0], f_EQ[-1])
)
ax.set_ylabel("Frequency (Hz)")
ax.set_title("MAHY32")

m = s3.get_manager()
data = m.get_segment(date3 - datetime.timedelta(seconds=DELTA_S), date3 + datetime.timedelta(seconds=DELTA_S))
f_EQ, t_EQ, spectro_EQ = make_spectrogram(
    data, m.sampling_f, t_res=0.25, f_res=1,
    return_bins=True, normalize=True, vmin=60, vmax=120
)
ax = axs[1]
im0 = ax.imshow(
    spectro_EQ[::-1], cmap="inferno", aspect="auto", origin="lower",
    extent=(-DELTA_S, DELTA_S, f_EQ[0], f_EQ[-1])
)
ax.set_ylabel("Frequency (Hz)")
ax.set_title("MAHY33")

m = s4.get_manager()
data = m.get_segment(date4 - datetime.timedelta(seconds=DELTA_S), date4 + datetime.timedelta(seconds=DELTA_S))
f_EQ, t_EQ, spectro_EQ = make_spectrogram(
    data, m.sampling_f, t_res=0.25, f_res=1,
    return_bins=True, normalize=True, vmin=60, vmax=120
)
ax = axs[2]
im0 = ax.imshow(
    spectro_EQ[::-1], cmap="inferno", aspect="auto", origin="lower",
    extent=(-DELTA_S, DELTA_S, f_EQ[0], f_EQ[-1])
)
ax.set_ylabel("Frequency (Hz)")
ax.set_xlabel(r"Time from detection (s)")
ax.set_title("MAHY34")



fig.tight_layout()
fig.patch.set_facecolor('xkcd:white')
plt.savefig(
    f'../../../../data/MAHY/figures/career_examples.pdf',
    dpi=500,
    bbox_inches='tight',
    pad_inches=0
)